In [1]:
import sys
import os

# Automatic search for project processing
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
if project_root not in sys.path:
    sys.path.append(project_root)

In [ ]:
from scripts.preprocess import file_to_pd

df = file_to_pd('../Data/output.csv')

from scripts.render import render_mpl_table

render_mpl_table(df.head(5))
# plt.savefig("img/table.png", bbox_inches="tight")

In [3]:
from scripts.preprocess import mlp_preprocess

train_loader, test_loader, scaler_X, scaler_y = mlp_preprocess(df)

In [4]:
from scripts.train_nn import train_model

model = train_model(train_loader=train_loader)

In [5]:
from scripts.predict import predict_single, find_min_J
from scripts.torch_wrapper import TorchModelWrapper

In [ ]:
wrapped_model = TorchModelWrapper(model)

result = predict_single(wrapped_model, [25.0, 0.0002, 0.065, 0.005], scaler_X, scaler_y)
print("J =", result)

In [ ]:
bounds = [(5, 50), (100e-6, 200e-6), (0.04, 0.08), (0.003, 0.01)]

optimal_parameters, optimal_j = find_min_J(wrapped_model, bounds, scaler_X, scaler_y, steps_per_float=20)

print("Optimal parameters =", optimal_parameters,'\n')
print("Optimal  J =", optimal_j)